In [5]:
import PyPDF2
import pandas as pd
import numpy as np
import sys, os
import json
import re

# root = r"C:\Users\Victor\Documents\Cursos\Python\Automatizacion\PDF"
root = r"../PDF/"
path = os.path.join(root, "PDF")

# Funcion que obtiene los archivos con su ruta y se encarga de clasificarlos en arreglos
def ArchivosPDF(root, path):
    Archivos_Acuse_Recepcion = []
    Archivos_Opinion_Cumplimiento = []
    Archivos_Acuse_Presentacion = []
    Archivos_Constancia = []
    Archivos_Diot = []
    Archivos_no_clasificados = []
    
    for path, subdirs, files in os.walk(root):
        for name in files:
            # print(os.path.join(path, name))
            if os.path.join(name).endswith(".pdf"):
                #Archivos Acuse Recepcion
                if os.path.join(name).startswith("acuse recepcion"):
                    Archivos_Acuse_Recepcion.append(os.path.join(path, name))
                elif os.path.join(name).startswith("opinion cumplimiento"):
                    Archivos_Opinion_Cumplimiento.append(os.path.join(path, name))
                elif os.path.join(name).startswith("acuse presentacion"):
                    Archivos_Acuse_Presentacion.append(os.path.join(path, name))
                elif os.path.join(name).startswith("constancia"):
                    Archivos_Constancia.append(os.path.join(path, name))
                elif os.path.join(name).startswith("diot"):
                    Archivos_Diot.append(os.path.join(path, name))
                else: 
                    Archivos_no_clasificados.append(os.path.join(path, name))
                    
    return Archivos_Acuse_Recepcion, Archivos_Opinion_Cumplimiento, Archivos_Acuse_Presentacion, Archivos_Constancia,Archivos_Diot, Archivos_no_clasificados


# -> Papelines de Extraccion 

# Varibales de PDF's'
# Se asigna el tipo de pdf a un variable

Archivos_Acuse_Recepcion = (ArchivosPDF(root, path)[0])         #--> Solo contiene 1 archivo
Archivos_Opinion_Cumplimiento = (ArchivosPDF(root, path)[1])    #--> Solo contiene 1 archivo
Archivos_Acuse_Presentacion = (ArchivosPDF(root, path)[2])      #--> °Contiene archivos Dummies y 1 original para pruebas
Archivos_Constancia = (ArchivosPDF(root, path)[3])              #--> Solo contiene 1 archivo
Archivos_Diot = (ArchivosPDF(root, path)[4])         #--> Solo contiene 1 archivo
Archivos_no_clasificados = (ArchivosPDF(root, path)[5])         #--> Solo contiene 1 archivo


print(len(Archivos_Acuse_Recepcion))
print(len(Archivos_Opinion_Cumplimiento))
print(len(Archivos_Acuse_Presentacion))
print(len(Archivos_Constancia))
print(len(Archivos_Diot))
print(len(Archivos_no_clasificados))




13
11
1
12
10
0


In [26]:
# Principal Functions Text Mining by PDF types
# -> Each function returns a numpy array with the result found but, if not found, defaults to NA
# -> Each function uses "txt" value containing all the text by per PDF iteration
# -> Each function should return the headers so, that they are recognized on the first line by Pandas


def exportDataAcusePresentacion(Archivos):
    array_docs = []
    for i in range(len(Archivos)):

        pdf = open(Archivos[i], 'rb')
        # print(Archivos[i])
        reader = PyPDF2.PdfReader(pdf)
        page = reader.pages
        texto = ''

        for pag in range(len(page)):
            # print("pdf: ", i+1, "pagina: ",pag+1)
            texto += reader._get_page(pag).extractText()

        print("PDF: ", i+1)
        txt = re.sub("\n", " ", texto)
        # print(txt) #Muestra el texto en de todo el PDF

        # Text Mining
        # DATOS GENERALES

        # rfc
        rfc = re.search("RFC:.(.*?\s)", txt)[0]
        # Fecha
        try:
            Fecha_y_hora_presentacion = re.search(
                "Fecha y hora de presentaci.n:.{11}", txt)[0]
        except:
            Fecha_y_hora_presentacion = "NA"

        # numero de operacion
        try:
            Num_de_Operacion = re.search(
                "N.mero de operaci.n:.(\d*?\s)", txt)[0]
        except:
            Num_de_Operacion = "NA"
        # periodo de la declaracion
        try:
            Periodo_de_declaracion = re.search(
                "Per.odo de la declaraci.n:\s.(.+?\s)", txt)[0]
        except:
            Periodo_de_declaracion = "NA"
        # Ejercicio
        try:
            Ejercicio = re.search("Ejercicio:\s.(.+?\s)", txt)[0]
        except:
            Ejercicio = "NA"
        # Denominación o razón social:

        try:
            RazonSocial = re.search("Hoja\d\sde\s\d.(.*?)Tipo", txt).group(1)
        except:
            pass

        # SELECCION LINEA DE CAPTURA
        try:
            total_a_pagar = re.search("total a pagar:(.+?\s)", txt)[0]
            Vigente_hasta = re.search("Vigente hasta:(.+?\s)", txt)[0]
            Linea_de_Captura = "Linea de Captura : " + \
                re.search("Línea de Captura:.(.*?)Importe", txt).group(1)
        except:
            total_a_pagar = "total a pagar: NA "
            Vigente_hasta = "Vigente hasta: NA "
            Linea_de_Captura = "Linea de Captura : NA "

        # data_fila = [rfc, Fecha_y_hora_presentacion, Num_de_Operacion, Periodo_de_declaracion, Ejercicio,  total_a_pagar, Vigente_hasta, Linea_de_Captura, Archivos[i]]
        data_fila = np.array([rfc, Fecha_y_hora_presentacion, Num_de_Operacion, Periodo_de_declaracion,
                             Ejercicio,  total_a_pagar, Vigente_hasta, Linea_de_Captura, Archivos[i]])
        array_docs.append(data_fila)

    return array_docs

def exportDataDiot(Archivos):
    array_docs = []
    for i in range(len(Archivos)):

        pdf = open(Archivos[i], 'rb')
        # print(Archivos[i])
        reader = PyPDF2.PdfReader(pdf)
        page = reader.pages
        texto = ''

        for pag in range(len(page)):
            # print("pdf: ", i+1, "pagina: ",pag+1)
            texto += reader._get_page(pag).extractText()

        print("PDF: ", i+1)
        txt = re.sub("\n", " ", texto)
        # print(txt) #Muestra el texto en de todo el PDF

        # Text Mining
        # DATOS GENERALES
        
        # Usuario
        try:
            usuario = re.search("Usuario:.(.+?\s)", txt)
        except:
            usuario= "NA"
        
        # Archivo
        try:
            Archivo_Recibido = re.search("Archivo Recibido:.(.+?\s)", txt)
        except:
            Archivo_Recibido= "NA"
            
        # Tamanio
        try:
            tamanio = re.search("Tama.o:.(.+?\s)", txt)
        except:
            tamanio= "NA"
        # Fecha de recepcion
        try:
            Fecha_Recepcion = re.search("Fecha de Recepci.n:.(.+?\s)", txt)
        except:
            Fecha_Recepcion= "NA"
        # Hora de recepcion
        try:
            Hora_Recepcion = re.search("Hora de Recepci.n:.(.+?\s)", txt)
        except:
            Hora_Recepcion= "NA"
        # Folio
        try:
            Folio = re.search("Folio de Recepci.n:.(.+?\s)", txt)
        except:
            Folio= "NA"
        
        data_fila = np.array([
            usuario,
            Archivo_Recibido,
            tamanio,
            Fecha_Recepcion,
            Hora_Recepcion,
            Folio,
            Archivos[i]
        ])
        array_docs.append(data_fila)
        
    return array_docs

# ACUSE RECEPCION
# datos = exportDataAcusePresentacion(Archivos_Acuse_Recepcion)
# # datos_array = np.asarray(datos)
# datos_array = datos
# df = pd.DataFrame(datos_array)


# DIOT
# datos_diot = exportDataDiot(Archivos_Diot)
# df = pd.DataFrame(datos_diot)
# df



PDF:  1
PDF:  2
PDF:  3
PDF:  4
PDF:  5
PDF:  6
PDF:  7
PDF:  8
PDF:  9
PDF:  10


,0,1,2,3,4,5,6
0,"<re.Match object; span=(225, 246), match='Usua...","<re.Match object; span=(246, 299), match='Arch...","<re.Match object; span=(299, 311), match='Tama...","<re.Match object; span=(317, 347), match='Fech...","<re.Match object; span=(347, 374), match='Hora...","<re.Match object; span=(374, 403), match='Foli...",../PDF/diot agrol 06-03-23.pdf
1,"<re.Match object; span=(226, 247), match='Usua...","<re.Match object; span=(247, 300), match='Arch...","<re.Match object; span=(300, 312), match='Tama...","<re.Match object; span=(318, 348), match='Fech...","<re.Match object; span=(348, 375), match='Hora...","<re.Match object; span=(375, 404), match='Foli...",../PDF/diot bricks 28-02-2023.pdf
2,"<re.Match object; span=(48, 69), match='Usuari...","<re.Match object; span=(69, 122), match='Archi...","<re.Match object; span=(122, 134), match='Tama...","<re.Match object; span=(140, 170), match='Fech...","<re.Match object; span=(170, 197), match='Hora...","<re.Match object; span=(197, 226), match='Foli...",../PDF/diot cuadras asesores 07-12-22.pdf
3,"<re.Match object; span=(225, 246), match='Usua...","<re.Match object; span=(246, 299), match='Arch...","<re.Match object; span=(299, 311), match='Tama...","<re.Match object; span=(317, 347), match='Fech...","<re.Match object; span=(347, 374), match='Hora...","<re.Match object; span=(374, 403), match='Foli...",../PDF/diot desoflex 27-02-23.pdf
4,"<re.Match object; span=(226, 247), match='Usua...","<re.Match object; span=(247, 300), match='Arch...","<re.Match object; span=(300, 312), match='Tama...","<re.Match object; span=(318, 348), match='Fech...","<re.Match object; span=(348, 375), match='Hora...","<re.Match object; span=(375, 404), match='Foli...",../PDF/diot diseños luna 28-02-23.pdf
5,"<re.Match object; span=(226, 248), match='Usua...","<re.Match object; span=(248, 301), match='Arch...","<re.Match object; span=(301, 313), match='Tama...","<re.Match object; span=(319, 349), match='Fech...","<re.Match object; span=(349, 376), match='Hora...","<re.Match object; span=(376, 405), match='Foli...",../PDF/diot hector jauregui rios 28-02-23.pdf
6,None,None,None,None,None,None,../PDF/diot hector jauregui rios 14-02-23.pdf
7,"<re.Match object; span=(226, 248), match='Usua...","<re.Match object; span=(248, 301), match='Arch...","<re.Match object; span=(301, 313), match='Tama...","<re.Match object; span=(319, 349), match='Fech...","<re.Match object; span=(349, 376), match='Hora...","<re.Match object; span=(376, 405), match='Foli...",../PDF/diot monica alejandra zarate losa 13-03...
8,"<re.Match object; span=(226, 248), match='Usua...","<re.Match object; span=(248, 301), match='Arch...","<re.Match object; span=(301, 313), match='Tama...","<re.Match object; span=(319, 349), match='Fech...","<re.Match object; span=(349, 376), match='Hora...","<re.Match object; span=(376, 405), match='Foli...",../PDF/diot pascual ibarra 28-02-23.pdf
9,"<re.Match object; span=(224, 246), match='Usua...","<re.Match object; span=(246, 299), match='Arch...","<re.Match object; span=(299, 311), match='Tama...","<re.Match object; span=(317, 347), match='Fech...","<re.Match object; span=(347, 374), match='Hora...","<re.Match object; span=(374, 403), match='Foli...",../PDF/diot rigoberto mora 01-03-23.pdf
